# Lab 4 - Predicción de supervivientes del Titanic.

El objetivo de este laboratorio es entrenar un clasificador binario para el dataset de la siguiente competición:

https://www.kaggle.com/c/titanic/overview

Se busca que la salida del modelo sea la probabilidad de supervivencia del pasajero segun los datos de entrada.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FCEIA-AAII/lab4/blob/main/lab4.ipynb)

## Preparación del entorno.

Si no estamos parados en el repo, clonar y cd al repo. Esto nos permite usar el mismo notebook tanto local como en Google Colab.

In [ ]:
import os

REPO_NAME = "lab4"
if REPO_NAME not in os.getcwd():
  if not os.path.exists(REPO_NAME):
    !git clone https://github.com/FCEIA-AAII/{REPO_NAME}.git
  os.chdir(REPO_NAME)


Importar librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

Establecer GPU por defecto en caso de estar disponible.

In [ ]:
# Configurar para que TensorFlow utilice la GPU por defecto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar para que TensorFlow asigne memoria dinámicamente
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Especificar la GPU por defecto
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Manejar error
        print(e)

## Análisis exploratorio

In [ ]:
# Cargamos el dataset

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(train_df.head())

Inspeccionamos cada columna del dataset para entender su significado y valores posibles.

In [ ]:
cols = train_df.columns

for col in cols:
    print("--------------------")
    print("Columna: ", col)
    print("Cantidad de valores nulos: ", train_df[col].isnull().sum())
    print("Cantidad de valores únicos: ", train_df[col].nunique())
    print("Tipo de dato: ", train_df[col].dtype)
    # Si no es numérica, continuar con la siguiente columna
    if train_df[col].dtype == "object":
        print("--------------------")
        continue
    # Si es una columna categórica, mostrar la cantidad de veces que aparece cada valor
    if train_df[col].nunique() < 10:
        print("Valores únicos: ", train_df[col].unique())
        print("Cantidad de veces que aparece cada valor: ", train_df[col].value_counts())
    else:
        # Si es una columna numérica, mostramos media, desvío estándar, mínimo, máximo...
        print("Media: ", train_df[col].mean())
        print("Desvío estándar: ", train_df[col].std())
        print("Mínimo: ", train_df[col].min())
        print("Máximo: ", train_df[col].max())
    print("--------------------")


Probamos un primer modelo con un subset de features `["Pclass", "sex", "Age"]`.

In [ ]:
# Defino un diccionario para definir el tipo de dato de cada columna
column_types = {
    "Sex": "categoric",
}

# Diccionario para almacenar media y desvío estándar de las columnas numéricas.
# Me va a servir para realizar predicciones en el futuro.
numeric_stats = {}

# Diccionario para almacenar los valores únicos de las columnas categóricas.
# Me va a servir para realizar predicciones en el futuro.
categoric_values = {}

preprocessed_train_df = pd.DataFrame()
preprocessed_test_df = pd.DataFrame()

# Preprocesamiento del set de entrenamiento
for col in column_types.keys():
    if column_types[col] == "categoric":
        num_classes = train_df[col].nunique()
        # Obtengo el one-hot encoding de la columna
        one_hot = pd.get_dummies(train_df[col], prefix=col, dtype=np.float32)
        # Agrego las columnas al dataset preprocesado
        preprocessed_train_df = pd.concat([preprocessed_train_df, one_hot], axis=1)
        # Almaceno los valores únicos
        categoric_values[col] = train_df[col].unique()
    else:
        # Normalizo la columna
        preprocessed_train_df[col] = (train_df[col] - train_df[col].mean()) / train_df[col].std()
        # Almaceno media y desvío estándar
        numeric_stats[col] = {
            "mean": train_df[col].mean(),
            "std": train_df[col].std()
        }

# Agrergo la columna "Survived" al dataset preprocesado
preprocessed_train_df["Survived"] = train_df["Survived"]

# Muestro las primeras filas del dataset preprocesado
print(preprocessed_train_df.head())

# Drop nan values
preprocessed_train_df = preprocessed_train_df.dropna()

# Defino los vectores X_train, y_train, X_test e y_test
X_train = preprocessed_train_df.drop("Survived", axis=1).values.astype(np.float32)
y_train = preprocessed_train_df["Survived"].values.astype(np.float32)

print(X_train.shape, y_train.shape)


Defino un primer modelo y lo entreno.

In [ ]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)